# TextAttack & AllenNLP 

This is an example of testing adversarial attacks from TextAttack on pretrained models provided by AllenNLP. 

In a few lines of code, we load a sentiment analysis model trained on the Stanford Sentiment Treebank and configure it with a TextAttack model wrapper. Then, we initialize the TextBugger attack and run the attack on a few samples from the SST-2 train set.

For more information on AllenNLP pre-trained models: https://docs.allennlp.org/v1.0.0rc3/tutorials/getting_started/using_pretrained_models/

For more information about the TextBugger attack: https://arxiv.org/abs/1812.05271

<table class="ta-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/drive/1Inpcc6AymahreBRkOpyQJd3sqOJagtNR?usp=sharing"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/QData/TextAttack/blob/master/docs/datasets_models/Example_2_allennlp.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

In [ ]:
!pip install allennlp allennlp_models textattack

In [ ]:
!pip install nlp pyarrow transformers --upgrade

In [ ]:
from allennlp.predictors import Predictor
import allennlp_models.classification

import textattack

class AllenNLPModel(textattack.models.wrappers.ModelWrapper):
    def __init__(self):
        self.predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/basic_stanford_sentiment_treebank-2020.06.09.tar.gz")

    def __call__(self, text_input_list):
        outputs = []
        for text_input in text_input_list:
        outputs.append(self.predictor.predict(sentence=text_input))
        # For each output, outputs['logits'] contains the logits where
        # index 0 corresponds to the positive and index 1 corresponds 
        # to the negative score. We reverse the outputs (by reverse slicing,
        # [::-1]) so that negative comes first and positive comes second.
        return [output['logits'][::-1] for output in outputs]

model_wrapper = AllenNLPModel()

In [ ]:
from textattack.datasets import HuggingFaceNlpDataset
from textattack.attack_recipes import TextBuggerLi2018

dataset = HuggingFaceNlpDataset("glue", "sst2", "train")
attack = TextBuggerLi2018(model_wrapper)

results = list(attack.attack_dataset(dataset, indices=range(20)))
for idx, result in enumerate(results):
    print(f'Result {idx}:')
    print(result.__str__(color_method='ansi'))
    print('\n\n')
print()